In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27510723
paper_name = 'kwon_koo_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data (1)

In [31]:
original_data1 = pd.read_excel('raw_data/425_2016_2579_MOESM3_ESM.xlsx', sheet_name='Sheet1', skiprows=2)

In [32]:
print('Original data dimensions: %d x %d' % (original_data1.shape))

Original data dimensions: 4528 x 5


In [33]:
original_data1.head()

,strain,LogRatio_BestTag_Zscore,gene,GO_function,GO_component
0,YHR030C,16.41,SLT2,MAP kinase activity,cytoplasm*
1,YJL095W,14.39,BCK1,MAP kinase kinase kinase activity,intracellular
2,YDR293C,12.70,SSD1,RNA binding,cytoplasm
3,YGR092W,11.69,DBF2,protein serine/threonine kinase activity,bud neck*
4,YGL084C,11.58,GUP1,O-acyltransferase activity,endoplasmic reticulum*


In [34]:
original_data1['orf'] = original_data1['strain'].astype(str)

In [35]:
# Eliminate all white spaces & capitalize
original_data1['orf'] = clean_orf(original_data1['orf'])

In [36]:
# Translate to ORFs 
original_data1['orf'] = translate_sc(original_data1['orf'], to='orf')

In [37]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orf'])
print(original_data1.loc[~t,])

Empty DataFrame
Columns: [strain, LogRatio_BestTag_Zscore, gene, GO_function, GO_component, orf]
Index: []


In [38]:
original_data1['data'] = -original_data1['LogRatio_BestTag_Zscore']

In [39]:
original_data1.set_index('orf', inplace=True)

In [40]:
original_data1 = original_data1[['data']].copy()

In [41]:
original_data1 = original_data1.groupby(original_data1.index).mean()

In [42]:
original_data1.shape

(4497, 1)

# Load & process data (2)

In [18]:
original_data2 = pd.read_excel('raw_data/het/Background Corrected Ratios And Robust Z-Scores (FD Scores)(all data).xlsx', sheet_name='Sheet1')

In [20]:
original_data2['orf'] = original_data2['strain'].astype(str)

In [21]:
# Eliminate all white spaces & capitalize
original_data2['orf'] = clean_orf(original_data2['orf'])

In [22]:
# Translate to ORFs 
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [23]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [strain, tagInfo, location, tagNames, ctrlVal, treatVal, ctrlVal_log2, treatVal_log2, treatVal_log2_bgThresh, LogRatio, besttag_index, usable_tags, Avg_Index_byTagInfo, LogRatio_Avg, LogRatio_BestTag, LogRatio_Avg_Zscore, LogRatio_BestTag_Zscore, gene, essential_gene, feature_type, GO_process, GO_function, GO_component, notes, Cross_Del_Count, Cross_Dels, description, orf]
Index: []

[0 rows x 28 columns]


In [26]:
original_data2['data'] = -pd.to_numeric(original_data2['LogRatio_BestTag_Zscore'], errors='coerce')

In [27]:
original_data2.set_index('orf', inplace=True)

In [28]:
original_data2 = original_data2[['data']].copy()

In [29]:
original_data2 = original_data2.groupby(original_data2.index).mean()

In [30]:
original_data2.shape

(1109, 1)

# Merge data

In [43]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [44]:
original_data.head()

,data_1,data_2
orf,,
YAL001C,NaN,0.67
YAL002W,0.07,NaN
YAL003W,NaN,0.00
YAL004W,0.26,NaN
YAL005C,0.00,NaN


# Prepare the final dataset

In [45]:
data = original_data.copy()

In [46]:
dataset_ids = [4826, 5181]
datasets = datasets.reindex(index=dataset_ids)

In [47]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [48]:
data.head()

dataset_id,4826,5181
data_type,value,value
orf,,
YAL001C,NaN,0.67
YAL002W,0.07,NaN
YAL003W,NaN,0.00
YAL004W,0.26,NaN
YAL005C,0.00,NaN


## Subset to the genes currently in SGD

In [49]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [50]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [51]:
data.head()

,dataset_id,4826,5181
,data_type,value,value
gene_id,orf,,
1,YAL001C,NaN,0.67
2,YAL002W,0.07,NaN
3,YAL003W,NaN,0.00
1863,YAL004W,0.26,NaN
4,YAL005C,0.00,NaN


# Normalize

In [52]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [53]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [54]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [55]:
data_all.head()


,dataset_id,4826,5181,4826,5181
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1,YAL001C,NaN,0.67,NaN,0.447197
2,YAL002W,0.07,NaN,0.024288,NaN
3,YAL003W,NaN,0.00,NaN,0.059907
1863,YAL004W,0.26,NaN,0.155194,NaN
4,YAL005C,0.00,NaN,-0.023940,NaN


# Print out

In [56]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [57]:
from IO.save_data_to_db3 import *

In [58]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 27510723...
Inserting the new data...


100%|██████████| 2/2 [00:16<00:00,  8.17s/it]

Updating the data_modified_on field...
